In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chef_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a best chef. you can create a easy recipes for me with gradients easy to find ",
        ),
        ("human", "I wanna make {cusine} food"),
    ]
)

chef_chain = chef_template | chat

In [ ]:
veg_chef_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)

veg_chef = veg_chef_template | chat

final_chain = {"recipe": chef_chain} | veg_chef

final_chain.invoke({"cusine": "indian"})